# Parameter Sweeps

## Overview
Systematic grid-search over the insurance parameter space using `ParameterSweeper`.  We demonstrate basic sweeps, optimal-region identification, pre-defined scenarios, heatmap analysis, and result export.

- **Prerequisites**: [optimization/01_optimization_overview](01_optimization_overview.ipynb)
- **Estimated runtime**: 2-4 minutes
- **Audience**: [Practitioner] / [Developer]

In [ ]:
"""Google Colab setup: mount Drive and install package dependencies.

Run this cell first. If prompted to restart the runtime, do so, then re-run all cells.
This cell is a no-op when running locally.
"""
import sys, os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    NOTEBOOK_DIR = '/content/drive/My Drive/Colab Notebooks/ei_notebooks/optimization'

    os.chdir(NOTEBOOK_DIR)
    if NOTEBOOK_DIR not in sys.path:
        sys.path.append(NOTEBOOK_DIR)

    !pip install ergodic-insurance -q 2>&1 | tail -3
    print('\nSetup complete. If you see numpy/scipy import errors below,')
    print('restart the runtime (Runtime > Restart runtime) and re-run all cells.')

## Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

from ergodic_insurance.config import ManufacturerConfig
from ergodic_insurance.manufacturer import WidgetManufacturer
from ergodic_insurance.parameter_sweep import ParameterSweeper, SweepConfig
from ergodic_insurance.business_optimizer import BusinessOptimizer

plt.style.use("seaborn-v0_8-darkgrid")
sns.set_palette("husl")

# Reproducibility
SEED = 42
np.random.seed(SEED)

## 1. Basic Parameter Sweep

Define a grid over company size, operating margin, and asset turnover, then sweep.

In [ ]:
config = ManufacturerConfig(
    initial_assets=10e6,
    asset_turnover_ratio=1.0,
    base_operating_margin=0.08,
    tax_rate=0.25,
    retention_ratio=0.6,
)
base_manufacturer = WidgetManufacturer(config)
optimizer = BusinessOptimizer(base_manufacturer)

sweeper = ParameterSweeper(
    optimizer=optimizer,
    cache_dir="./cache/parameter_sweeps",
    use_parallel=True,
)

simple_config = SweepConfig(
    parameters={
        "initial_assets": [1e6, 5e6, 10e6, 20e6],
        "base_operating_margin": [0.05, 0.08, 0.10, 0.12],
        "asset_turnover": [0.8, 1.0, 1.2],
    },
    fixed_params={
        "time_horizon": 10,
        "n_simulations": 1000,
        "max_risk_tolerance": 0.01,
        "min_roe_threshold": 0.10,
    },
    n_workers=4,
    batch_size=10,
)

print(f"Total combinations: {len(simple_config.generate_grid())}")
print(f"Estimated runtime : {simple_config.estimate_runtime(seconds_per_run=2.0)}")

In [ ]:
print("Starting parameter sweep...")
results = sweeper.sweep(simple_config)
print(f"Sweep complete: {len(results)} results")
results.head()

## 2. Optimal Region Identification

Filter to configurations with < 1% bankruptcy risk and select the top 20% by ROE.

In [ ]:
optimal_results, param_summary = sweeper.find_optimal_regions(
    results,
    objective="optimal_roe",
    constraints={"ruin_probability": (0, 0.01)},
    top_percentile=80,
)

print(f"Optimal configurations: {len(optimal_results)}")
print(f"\nParameter statistics for optimal region:")
print(param_summary)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(results["optimal_roe"].dropna(), bins=30, alpha=0.5, label="All", color="blue")
axes[0].hist(optimal_results["optimal_roe"], bins=20, alpha=0.7, label="Optimal", color="red")
axes[0].set_xlabel("Optimal ROE")
axes[0].set_ylabel("Frequency")
axes[0].set_title("ROE Distribution: All vs. Optimal")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].scatter(results["ruin_probability"], results["optimal_roe"],
                alpha=0.5, s=50, label="All")
axes[1].scatter(optimal_results["ruin_probability"], optimal_results["optimal_roe"],
                color="red", alpha=0.7, s=50, label="Optimal")
axes[1].axvline(x=0.01, color="black", ls="--", alpha=0.5, label="Risk Constraint")
axes[1].set_xlabel("Bankruptcy Risk")
axes[1].set_ylabel("Optimal ROE")
axes[1].set_title("Risk-Return Trade-off")
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Pre-defined Scenarios

The sweeper ships with ready-made scenario configurations.

In [ ]:
scenarios = sweeper.create_scenarios()

print("Available scenarios:")
for name, cfg in scenarios.items():
    n = len(cfg.generate_grid())
    print(f"  {name}: {list(cfg.parameters.keys())} ({n} combos)")

In [ ]:
company_sizes_config = scenarios["company_sizes"]
company_sizes_config.n_simulations = 500

print("Running company-sizes scenario...")
company_results = sweeper.sweep(company_sizes_config)
print(f"Generated {len(company_results)} results")

size_analysis = company_results.groupby("initial_assets").agg(
    {"optimal_roe": ["mean", "std"], "ruin_probability": ["mean", "std"]}
)
print("\nAnalysis by company size:")
print(size_analysis)

## 4. Heatmap Analysis

Pivot the sweep results into heatmaps to spot patterns.

In [ ]:
pivot_roe = company_results.pivot_table(
    values="optimal_roe", index="base_operating_margin",
    columns="initial_assets", aggfunc="mean",
)
pivot_risk = company_results.pivot_table(
    values="ruin_probability", index="base_operating_margin",
    columns="initial_assets", aggfunc="mean",
)

fig, axes = plt.subplots(1, 2, figsize=(15, 5))
sns.heatmap(pivot_roe, annot=True, fmt=".3f", cmap="RdYlGn",
            cbar_kws={"label": "Optimal ROE"}, ax=axes[0])
axes[0].set_title("ROE by Size and Margin")

sns.heatmap(pivot_risk, annot=True, fmt=".4f", cmap="RdYlGn_r",
            cbar_kws={"label": "Bankruptcy Risk"}, ax=axes[1])
axes[1].set_title("Bankruptcy Risk by Size and Margin")

plt.tight_layout()
plt.show()

## 5. Market Conditions Scenario

Sweep premium-loading levels to see how soft / hard markets shift optimal strategies.

In [ ]:
market_config = scenarios["market_conditions"]
market_config.fixed_params["n_simulations"] = 500

print("Running market-conditions scenario...")
market_results = sweeper.sweep(market_config)
print(f"Generated {len(market_results)} results")

fig, axes = plt.subplots(1, 3, figsize=(15, 4))
for ax, col, title in zip(axes,
    ["optimal_roe", "total_premium", "optimal_retention"],
    ["ROE", "Total Premium ($)", "Optimal Retention"],
):
    market_results.boxplot(column=col, by="premium_loading", ax=ax)
    ax.set_title(title)
    ax.set_xlabel("Premium Loading")
    ax.grid(True, alpha=0.3)

plt.suptitle("Impact of Market Conditions", fontweight="bold", y=1.02)
plt.tight_layout()
plt.show()

## Key Takeaways

- **Grid search** systematically maps the parameter landscape and reveals non-obvious optimal regions.
- **Optimal-region filtering** (e.g., top 20% ROE with < 1% ruin) quickly narrows the search.
- **Heatmaps** expose interactions between company size and operating margin that univariate sweeps miss.
- In **hard markets**, optimal retentions rise and ROE compresses -- the parameter sweep quantifies how much.

## Next Steps

- [optimization/02_sensitivity_analysis](02_sensitivity_analysis.ipynb) -- tornado diagrams and two-way analysis
- [optimization/04_retention_optimization](04_retention_optimization.ipynb) -- deep-dive into deductible optimization
- [optimization/06_state_driven_exposures](06_state_driven_exposures.ipynb) -- exposures that track real financial state